<a href="https://colab.research.google.com/github/vivekpatidar1413/ML_bootcamp_vivek/blob/Neural-Network/FINAL_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the MNIST dataset


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import random

In [ ]:
train_df = pd.read_csv("/content/sample_data/mnist_train_small.csv")
test_df  = pd.read_csv("/content/sample_data/mnist_test.csv")

In [ ]:
train = train_df.to_numpy()
test  = test_df.to_numpy()

In [ ]:
# splitting the dataset into 'lables' as y and 'feature (784pixels) as x'
x_train = train[:,1:]/255
y_train = train[:,:1]

x_test = test[:,1:]/255
y_test = test[:,:1]

In [ ]:
#converting train_label in one hot encoder representation 

y_n_train=np.zeros((19999,10))

for i in range(19999):
  for j in range(10):
    if y_train[i] == j:
      y_n_train[i,j] = 1

In [ ]:
y_n_test=np.zeros((9999,10))

for i in range(9999):
  for j in range(10):
    if y_test[i] == j:
      y_n_test[i,j] = 1

In [ ]:
print(x_train.shape,y_n_train.shape,x_test.shape,y_n_test.shape)

(19999, 784) (19999, 10) (9999, 784) (9999, 10)


# INITIALIZATION


Some Useful Function 

In [ ]:
def sigmoid(z):
  temp = 1 / (1+np.exp(-z))
  return temp

In [ ]:
def sigmoid_nxt(z):
  return (np.exp(-z))/((np.exp(-z)+1)**2)

In [ ]:
def softmax(x):
  exps = np.exp(x - x.max())
  return exps / np.sum(exps, axis=0)

In [ ]:
def sigmoid_prime(x):
    return x*(1-x)

Our Neural Network has 4 layer amoung which two hidden layers have 100 and 50 respective units

### **784 -- 100 -- 50 -- 10**

the architecture of this network is shown above

# Forward propagation

In [ ]:
def forward_propagation (W1,W2,W3,X):
  Z1 = np.dot(W1,X)
  A1 = sigmoid(Z1)

  Z2 = np.dot(W2,A1)
  A2 = sigmoid(Z2)

  Z3 = np.dot(W3,A2)
  A3 = softmax(Z3)              #chk

  return A1,A2,A3,Z1,Z2,Z3

# backward propagation Algorithm

*   with updating the variables (parameters)






In [ ]:
def back_propagation (W1,W2,W3,A1,A2,A3,Z1,Z2,Z3,Y):

  # delta_l3 = (A3 - Y)*sigmoid_prime(A3)          
  # delta_l2 = delta_l3.dot(W3)*sigmoid_prime(A2) 
  # delta_l1 = delta_l2.dot(W2)*sigmoid_prime(A1)

  # delta_l3 = np.dot(error,A3)      
  '''
  error = A3 - Y
  delta_l3 = np.dot(error,A3)
  
  error = np.multiply(np.dot(W3.T,error),sigmoid_nxt(Z2))    #changed
  delta_l2 = np.dot(error,A2)

  error = np.multiply(np.dot(W2.T,error),sigmoid_nxt(Z1))    #changed
  delta_l1 = np.dot(error,A1)
  '''

  delta_l3 = A3 - Y
  delta_l2 = np.multiply(np.dot(W3.T,delta_l3),sigmoid_nxt(Z2))
  delta_l1 = np.multiply(np.dot(W2.T,delta_l2),sigmoid_nxt(Z1))

  return delta_l1,delta_l2,delta_l3

Updating the parameters

In [ ]:
def update_parameters(A1,A2,A3,delta_l1,delta_l2,delta_l3,X):
    
    '''learning rate'''

    alpha = 0.0001     
  
    Del_1 = np.zeros((100,784))
    Del_2 = np.zeros((50,100))
    Del_3 = np.zeros((10,50))
    
    for i in range(100):
        Del_1[i] += X*delta_l1[i]
        
    for i in range(50):
        Del_2[i] += A1*delta_l2[i]

    for i in range(10):
        Del_3[i] += A2*delta_l3[i]

      

    return Del_1,Del_2,Del_3

In [ ]:
def compute_accuracy(W1,W2,W3,x_val,y_val):
    
    t = 0
    for x, y in zip(x_val, y_val):
      a,b,c,p,q,r = forward_propagation(W1,W2,W3,x)
      output = c
      
      if np.argmax(output) == np.argmax(y):
        t+=1

    return t

# THE MAIN  FUNTION

calculating the cost by two methods

In [ ]:
def compute_cost(AL, Y):
    
    # m = Y.shape[1]
    # cost = (-1 / m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL)))
    # #print("cost="+str(cost))
    # return cost
    m =19999
    cost = 0
    for i in range(19999):
        cost += np.sum(Y[i]*np.log(AL[i]) + (1-Y[i])*np.log(1-AL[i]))

    return -1/m*(cost)

**THE MAIN TRAINING LOOP BEGINS FROM HERE**

In [ ]:
def train(x_train,y_n_train,iter):
  t = 0
  alpha = 0.00001
  m = 19999

  W1 = np.random.randn(100,784)*np.sqrt(1. / 100)
  W2 = np.random.randn(50,100)*np.sqrt(1. / 50)
  W3 = np.random.randn(10,50)*np.sqrt(1. / 10)
  
  parameter = {
      'W1': W1,
      'W2': W2,
      'W3': W3
  }
  
  for iteration in range(1,1+iter):
    prediction = []
    for X,Y in zip(x_train,y_n_train):
      
      A1,A2,A3,Z1,Z2,Z3 = forward_propagation(W1,W2,W3,X)
      
      change_W1,change_W2,change_W3 = back_propagation (W1,W2,W3,A1,A2,A3,Z1,Z2,Z3,Y)

      a,b,c = update_parameters(A1,A2,A3,change_W1,change_W2,change_W3,X)

      W1 = W1 - alpha*a/m
      W2 = W2 - alpha*b/m
      W3 = W3 - alpha*c/m

      prediction.append(A3)

    accuracy = compute_accuracy(a,b,c,x_test, y_n_test)

    print('iteration',iteration,'gives cost =',compute_cost(prediction,y_n_train))
    print('accuracy is ',accuracy,'out of 9999 are correcctly pridicted')

Defining the Parameters

Cutting down the array for smaller demension for testing

In [ ]:
X1 = x_train[:19999]
Y1 = y_n_train[:19999]

In [ ]:
train(X1,Y1,10)

iteration 1 gives cost = 3.7636694262259693
accuracy is  958 out of 9999 are correcctly pridicted
iteration 2 gives cost = 3.763647668035507
accuracy is  958 out of 9999 are correcctly pridicted
iteration 3 gives cost = 3.7636259110674533
accuracy is  958 out of 9999 are correcctly pridicted
iteration 4 gives cost = 3.7636041553218083
accuracy is  958 out of 9999 are correcctly pridicted
iteration 5 gives cost = 3.763582400798333
accuracy is  958 out of 9999 are correcctly pridicted
iteration 6 gives cost = 3.763560647496976
accuracy is  958 out of 9999 are correcctly pridicted
iteration 7 gives cost = 3.7635388954176725
accuracy is  958 out of 9999 are correcctly pridicted
iteration 8 gives cost = 3.7635171445603004
accuracy is  958 out of 9999 are correcctly pridicted
iteration 9 gives cost = 3.763495394924761
accuracy is  958 out of 9999 are correcctly pridicted
iteration 10 gives cost = 3.7634736465109606
accuracy is  958 out of 9999 are correcctly pridicted


# Using Sklearn


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

mlp = MLPClassifier(hidden_layer_sizes=(100,50),max_iter=30)
mlp.fit(x_train,y_n_train)

prediction = mlp.predict(x_test)

accuracy = accuracy_score(prediction,y_n_test)

print('the accuracy obtained by sklearn model is ',accuracy,'%')

the accuracy obtained by sklearn model is  0.9477947794779478 %


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
